In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q ftp://mirror.klaus-uwe.me/apache/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-2.4.7-bin-hadoop2.7.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Test the spark
df = spark.createDataFrame([{"hello": "world"} for x in range(1000)])
df.show(3, False)

/content/spark-2.4.7-bin-hadoop2.7/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


+-----+
|hello|
+-----+
|world|
|world|
|world|
+-----+
only showing top 3 rows



In [ ]:
# Check the pyspark version
import pyspark
print(pyspark.__version__)

2.4.7


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [ ]:
from pyspark.ml.clustering import KMeans

dataset = spark.read.csv("Book1.csv",header=True,inferSchema=True)

dataset.show()


+--------------+--------------------+--------------+--------------------+-----------+------+--------------------+-----------------+--------------+--------------+--------------------+-----------------------------------------+-----------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+---------+
|Patient Number|State Patient Number|Date Announced|Estimated Onset Date|Age Bracket|Gender|       Detected City|Detected District|Detected State|Current Status|               Notes|Contracted from which Patient (Suspected)|Nationality|Type of transmission|Status Change Date|            Source_1|            Source_2|            Source_3|        Backup Notes|Num Cases|
+--------------+--------------------+--------------+--------------------+-----------+------+--------------------+-----------------+--------------+--------------+--------------------+-----------------------------------------+-----------+--------------------+-

In [ ]:
type(dataset)

pyspark.sql.dataframe.DataFrame

In [ ]:
dataset.head()

Row(Patient Number=1, State Patient Number='KL-TS-P1', Date Announced='1/30/2020', Estimated Onset Date=None, Age Bracket=20, Gender='F', Detected City='Thrissur', Detected District='Thrissur', Detected State='Kerala', Current Status='Recovered', Notes='Travelled from Wuhan', Contracted from which Patient (Suspected)='P6', Nationality='India', Type of transmission='Imported', Status Change Date='2/14/2020', Source_1='https://twitter.com/vijayanpinarayi/status/1222819465143832577', Source_2='https://weather.com/en-IN/india/news/news/2020-02-14-kerala-defeats-coronavirus-indias-three-covid-19-patients-successfully', Source_3='https://weather.com/en-IN/india/news/news/2020-02-14-kerala-defeats-coronavirus-indias-three-covid-19-patients-successfully', Backup Notes='Student from Wuhan', Num Cases=1)

In [ ]:
 dataset.collect()


[Row(Patient Number=1, State Patient Number='KL-TS-P1', Date Announced='1/30/2020', Estimated Onset Date=None, Age Bracket=20, Gender='F', Detected City='Thrissur', Detected District='Thrissur', Detected State='Kerala', Current Status='Recovered', Notes='Travelled from Wuhan', Contracted from which Patient (Suspected)='P6', Nationality='India', Type of transmission='Imported', Status Change Date='2/14/2020', Source_1='https://twitter.com/vijayanpinarayi/status/1222819465143832577', Source_2='https://weather.com/en-IN/india/news/news/2020-02-14-kerala-defeats-coronavirus-indias-three-covid-19-patients-successfully', Source_3='https://weather.com/en-IN/india/news/news/2020-02-14-kerala-defeats-coronavirus-indias-three-covid-19-patients-successfully', Backup Notes='Student from Wuhan', Num Cases=1),
 Row(Patient Number=2, State Patient Number='KL-AL-P1', Date Announced='2/2/2020', Estimated Onset Date=None, Age Bracket=45, Gender='M', Detected City='Alappuzha', Detected District='Alappuzh

In [ ]:
dataset.describe().show()

+-------+-----------------+--------------------+--------------+--------------------+------------------+------+-------------+-----------------+--------------+--------------+--------------------+-----------------------------------------+-----------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+---------+
|summary|   Patient Number|State Patient Number|Date Announced|Estimated Onset Date|       Age Bracket|Gender|Detected City|Detected District|Detected State|Current Status|               Notes|Contracted from which Patient (Suspected)|Nationality|Type of transmission|Status Change Date|            Source_1|            Source_2|            Source_3|        Backup Notes|Num Cases|
+-------+-----------------+--------------------+--------------+--------------------+------------------+------+-------------+-----------------+--------------+--------------+--------------------+-----------------------------------------+-

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
dataset.columns

['Patient Number',
 'State Patient Number',
 'Date Announced',
 'Estimated Onset Date',
 'Age Bracket',
 'Gender',
 'Detected City',
 'Detected District',
 'Detected State',
 'Current Status',
 'Notes',
 'Contracted from which Patient (Suspected)',
 'Nationality',
 'Type of transmission',
 'Status Change Date',
 'Source_1',
 'Source_2',
 'Source_3',
 'Backup Notes',
 'Num Cases']

In [ ]:
string_indexer = StringIndexer(inputCol ='Age Bracket', outputCol='features1')

In [ ]:
final_data = string_indexer.fit(dataset).transform(dataset)
final_data.show()

+--------------+--------------------+--------------+--------------------+-----------+------+--------------------+-----------------+--------------+--------------+--------------------+-----------------------------------------+-----------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+---------+---------+
|Patient Number|State Patient Number|Date Announced|Estimated Onset Date|Age Bracket|Gender|       Detected City|Detected District|Detected State|Current Status|               Notes|Contracted from which Patient (Suspected)|Nationality|Type of transmission|Status Change Date|            Source_1|            Source_2|            Source_3|        Backup Notes|Num Cases|features1|
+--------------+--------------------+--------------+--------------------+-----------+------+--------------------+-----------------+--------------+--------------+--------------------+-----------------------------------------+-----------+--

In [ ]:
from pyspark.ml.feature import OneHotEncoderEstimator

encoder = OneHotEncoderEstimator(
    inputCols=['Age Bracket'],  
    outputCols=['Hello']
)

model = encoder.fit(final_data)
encoded = model.transform(final_data)
encoded.show()

+--------------+--------------------+--------------+--------------------+-----------+------+--------------------+-----------------+--------------+--------------+--------------------+-----------------------------------------+-----------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+---------+---------+---------------+
|Patient Number|State Patient Number|Date Announced|Estimated Onset Date|Age Bracket|Gender|       Detected City|Detected District|Detected State|Current Status|               Notes|Contracted from which Patient (Suspected)|Nationality|Type of transmission|Status Change Date|            Source_1|            Source_2|            Source_3|        Backup Notes|Num Cases|features1|          Hello|
+--------------+--------------------+--------------+--------------------+-----------+------+--------------------+-----------------+--------------+--------------+--------------------+------------------------

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
dataset.columns

['Patient Number',
 'State Patient Number',
 'Date Announced',
 'Estimated Onset Date',
 'Age Bracket',
 'Gender',
 'Detected City',
 'Detected District',
 'Detected State',
 'Current Status',
 'Notes',
 'Contracted from which Patient (Suspected)',
 'Nationality',
 'Type of transmission',
 'Status Change Date',
 'Source_1',
 'Source_2',
 'Source_3',
 'Backup Notes',
 'Num Cases']

In [ ]:
dataset.printSchema()

root
 |-- Patient Number: integer (nullable = true)
 |-- State Patient Number: string (nullable = true)
 |-- Date Announced: string (nullable = true)
 |-- Estimated Onset Date: string (nullable = true)
 |-- Age Bracket: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Detected City: string (nullable = true)
 |-- Detected District: string (nullable = true)
 |-- Detected State: string (nullable = true)
 |-- Current Status: string (nullable = true)
 |-- Notes: string (nullable = true)
 |-- Contracted from which Patient (Suspected): string (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Type of transmission: string (nullable = true)
 |-- Status Change Date: string (nullable = true)
 |-- Source_1: string (nullable = true)
 |-- Source_2: string (nullable = true)
 |-- Source_3: string (nullable = true)
 |-- Backup Notes: string (nullable = true)
 |-- Num Cases: integer (nullable = true)



In [ ]:
from pyspark.sql.types import IntegerType

dataset = dataset.withColumn('Detected City',dataset['Detected City'].cast(IntegerType()))
dataset = dataset.withColumn('Num Cases',dataset['Num Cases'].cast(IntegerType()))
dataset = dataset.withColumn('Patient Number',dataset['Patient Number'].cast(IntegerType()))
dataset = dataset.withColumn('Age Bracket',dataset['Age Bracket'].cast(IntegerType()))
dataset = dataset.withColumn('Contracted from which Patient (Suspected)',dataset['Contracted from which Patient (Suspected)'].cast(IntegerType()))


In [ ]:
dataset.printSchema()

root
 |-- Patient Number: integer (nullable = true)
 |-- State Patient Number: string (nullable = true)
 |-- Date Announced: string (nullable = true)
 |-- Estimated Onset Date: string (nullable = true)
 |-- Age Bracket: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Detected City: integer (nullable = true)
 |-- Detected District: string (nullable = true)
 |-- Detected State: string (nullable = true)
 |-- Current Status: string (nullable = true)
 |-- Notes: string (nullable = true)
 |-- Contracted from which Patient (Suspected): integer (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Type of transmission: string (nullable = true)
 |-- Status Change Date: string (nullable = true)
 |-- Source_1: string (nullable = true)
 |-- Source_2: string (nullable = true)
 |-- Source_3: string (nullable = true)
 |-- Backup Notes: string (nullable = true)
 |-- Num Cases: integer (nullable = true)



In [ ]:
dataset.show()

+--------------+--------------------+--------------+--------------------+-----------+------+-------------+-----------------+--------------+--------------+--------------------+-----------------------------------------+-----------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+---------+
|Patient Number|State Patient Number|Date Announced|Estimated Onset Date|Age Bracket|Gender|Detected City|Detected District|Detected State|Current Status|               Notes|Contracted from which Patient (Suspected)|Nationality|Type of transmission|Status Change Date|            Source_1|            Source_2|            Source_3|        Backup Notes|Num Cases|
+--------------+--------------------+--------------+--------------------+-----------+------+-------------+-----------------+--------------+--------------+--------------------+-----------------------------------------+-----------+--------------------+------------------+---

In [ ]:
vec_assembler = VectorAssembler(inputCols = ('Age Bracket','Patient Number','Num Cases'), outputCol='features')
#vec_assembler = VectorAssembler(inputCols = ('Patient_Number'), outputCol = 'features')

In [ ]:
final_data = vec_assembler.transform(dataset)


In [ ]:
from pyspark.ml.feature import StandardScaler

In [ ]:
scaler = StandardScaler(inputCol ='features', outputCol = 'scaledFeatures', withStd = True, withMean = False)

In [ ]:
# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(final_data)

In [ ]:
# Normalize each feature to have unit standard deviation.
final_data = scalerModel.transform(final_data)
final_data.show()

+--------------+--------------------+--------------+--------------------+-----------+------+-------------+-----------------+--------------+--------------+--------------------+-----------------------------------------+-----------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+---------+---------------+--------------------+
|Patient Number|State Patient Number|Date Announced|Estimated Onset Date|Age Bracket|Gender|Detected City|Detected District|Detected State|Current Status|               Notes|Contracted from which Patient (Suspected)|Nationality|Type of transmission|Status Change Date|            Source_1|            Source_2|            Source_3|        Backup Notes|Num Cases|       features|      scaledFeatures|
+--------------+--------------------+--------------+--------------------+-----------+------+-------------+-----------------+--------------+--------------+--------------------+-----------------------

In [ ]:
# Training a k-means model.
kmeans = KMeans(featuresCol='scaledFeatures',k=3)
model = kmeans.fit(final_data)

In [ ]:
# Evaluate clustering by computing Within Set Sum of Squared Errors.
wssse = model.computeCost(final_data)
print("Within Set Sum of Squared Errors = " + str(wssse))

Within Set Sum of Squared Errors = 25.39660262662114


In [ ]:

centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[3.2545002  0.83550442 0.        ]
[3.53292816 2.50651325 0.        ]
[4.82349553 0.58976782 0.        ]


In [ ]:
model.transform(final_data).select('prediction').show()

+----------+
|prediction|
+----------+
|         0|
|         0|
|         0|
|         2|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
|         1|
+----------+
only showing top 20 rows

